In [1]:
import mesa
import numpy as np
from enum import Enum
import uuid
import random

def new_uuid():
    # Generate a UUID
    unique_id = uuid.uuid4()

    # Convert the UUID to an integer
    int_uuid = int(unique_id.int)

    return int_uuid

In [2]:
class FireAgent(mesa.Agent):
    def __init__(self, unique_id, model, state: int):
        super().__init__(unique_id, model)
        # Agent może być w jednym z 6 stanów:
        # 0 - brak pożaru
        # 1 - wczesny ogień
        # 2 - średni ogień
        # 3 - pełny ogień
        # 4 - ekstremalny ogień
        # 5 - obszar spalony (obszar spalony nie może podpalić się po raz kolejny, gdyż całe paliwo zostało spalone) 
        self.state = state
        self.time_step = 0
        self.type = "Fire"

    def extinguised(self):
        if self.time_step <= 0:
            self.time_step = 0
            self.state = 0
            return True
        else:
            return False
        
    def step(self):
        if 0 < self.state < 5:
            self.time_step += 0.5
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        firefighting_units = 0
        for c in cellmates:
            if c.type == "FireFighter":
                firefighting_units += 1
                
        if self.state == 1:
            self.time_step -= firefighting_units*2
            if self.extinguised():
                return
            if self.time_step >= 20:
                self.time_step = 0
                self.state = 2
                
        elif self.state == 2:
            self.time_step -= firefighting_units
            if self.extinguised():
                return
            if self.time_step >= 10:
                self.time_step = 0
                self.state = 3
                
        elif self.state == 3:
            self.time_step -= firefighting_units*0.5
            if self.extinguised():
                return
            neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            for n in neighborhood:
                agents_n = self.model.grid.get_cell_list_contents([(n[0], n[1])])
                break_search = False
                for c in agents_n:
                    if c.type == "Fire" and c.state == 0:
                        c.state = 1
                        c.time_step = 0
                        break_search = True
                        break
                if break_search:
                    break
            if self.time_step >= 10:
                self.time_step = 0
                self.state = 4
                
        elif self.state == 4:
            self.time_step -= firefighting_units*0.25
            if self.extinguised():
                return
            neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            for n in neighborhood:
                agents_n = self.model.grid.get_cell_list_contents([(n[0], n[1])])
                for c in cellmates:
                    if c.type == "Fire" and c.state < 4:
                        c.state += 1
                        c.time_step = 0
                        break
            if self.time_step >= 10:
                self.time_step = 0
                self.state = 5

class FireFighterAgent(mesa.Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.type = "FireFighter"
        
    def step(self):
        pass

In [3]:
class FireControllerAgentModel(mesa.Model):
    def __init__(self, width, height, fire_fighters, fire_agents_values: list):
        """
        width, height - rozmiary lasu (dla symulacji przyjmujemy rozmiar 10x10
        
        fire_fighters - ilość jednostek straży pożarnej, jakie FCA ma do dyspozycji
        
        fire_agents_values - lista wartości ze zbioru {1,2,3,4} określająca początkowe wartości pożaru w losowo wybranych komórkach.
        Maksymalna długość listy to 100 (tyle ile mamy komórek w siatce 10x10). Przykładowo, dla listy [1,1,3,2], cztery losowo 
        wybrane komórki zostaną zainicjalizowane właśnie takimi wartościami poziomu pożaru
        """
        self.width = width
        self.height = height
        self.fire_fighters = fire_fighters
        self.grid = mesa.space.MultiGrid(width, height, False)
        self.schedule = mesa.time.RandomActivation(self)

        total_cells = width * height
        all_indices = [(i, j) for i in range(width) for j in range(height)]
        fire_cells = random.sample(all_indices, len(fire_agents_values))
        no_fire_cells = [index for index in all_indices if index not in fire_cells]
        for i, index in enumerate(fire_cells):
            a = FireAgent(new_uuid(), self, fire_agents_values[i])
            self.grid.place_agent(a, index)
            self.schedule.add(a)
        for i, index in enumerate(no_fire_cells):
            a = FireAgent(new_uuid(), self, 0)
            self.grid.place_agent(a, index)
            self.schedule.add(a)
        self.assign_firefighters_to_grid()

    def step(self):
        self.schedule.step()
        self.assign_firefighters_to_grid()

    def get_total_fire_level(self):
        result = 0
        for x in range(self.width):
            for y in range(self.height):
                fire = self.get_fire_agent(x, y)
                result += fire.state if fire.state < 5 else 0
        return result

    def get_total_burned_level(self):
        result = 0
        for x in range(self.width):
            for y in range(self.height):
                fire = self.get_fire_agent(x, y)
                result += fire.state if fire.state == 5 else 0
        return result

    def get_fire_agent(self, x, y):
        agents = self.grid.get_cell_list_contents([(x,y)])
        for a in agents:
            if a.type == "Fire":
                return a

    def delete_firefighters(self):
        for x in range(self.width):
            for y in range(self.height):
                agents = self.grid.get_cell_list_contents([(x,y)])
                for a in agents:
                    if a.type == "FireFighter":
                        self.schedule.remove(a)
                        self.grid.remove_agent(a)
        
    def assign_firefighters_to_grid(self):
        self.delete_firefighters()
        values = np.zeros((self.width, self.height))
        for x in range(self.width):
            for y in range(self.height):
                fire = self.get_fire_agent(x, y)
                values[x,y] = fire.state if fire.state < 5 else 0
                
        # Get the indices in descending order of values
        indices = np.argsort(values.ravel())[::-1]
        # Convert the 1D indices to 2D indices
        row_indices, col_indices = np.unravel_index(indices, values.shape)
        # Filter indices where the value is greater than zero
        non_zero_indices = (values[row_indices, col_indices] > 0).nonzero()
        # Get the final indices
        sorted_indices = []
        for x, y in zip(row_indices[non_zero_indices], col_indices[non_zero_indices]):
            sorted_indices.append((x,y))

        available_firefighters = self.fire_fighters
        for index in sorted_indices:
            x, y = index
            if self.get_fire_agent(x,y).state == 4:
                new_firefighters = min(3, available_firefighters)
                for _ in range(new_firefighters):
                    a = FireFighterAgent(new_uuid(), self)
                    self.grid.place_agent(a, (x, y))
                    self.schedule.add(a)
                available_firefighters -= new_firefighters
            elif self.get_fire_agent(x,y).state == 3:
                new_firefighters = min(2, available_firefighters)
                for _ in range(new_firefighters):
                    a = FireFighterAgent(new_uuid(), self)
                    self.grid.place_agent(a, (x, y))
                    self.schedule.add(a)
                available_firefighters -= new_firefighters
            else:
                new_firefighters = min(1, available_firefighters)
                for _ in range(new_firefighters):
                    a = FireFighterAgent(new_uuid(), self)
                    self.grid.place_agent(a, (x, y))
                    self.schedule.add(a)
                available_firefighters -= new_firefighters
            if available_firefighters == 0:
                break
        

In [5]:
starter_model = FireControllerAgentModel(width=10, height=10, fire_fighters=1, fire_agents_values=[1])

for i in range(10):
    starter_model.step()
    print(starter_model.get_total_fire_level())

print(f"Burned {starter_model.get_total_burned_level()}")

0
0
0
0
0
0
0
0
0
0
Burned 0


In [6]:
class QLearning():
    def __init__(self):
        self.width = 10
        self.height = 10
        self.fire_fighters = 20
        self.actions = [i+1 for i in range(20)]
        self.states = [20*(i+1) for i in range(25)]
        self.q_table = np.zeros((len(self.states), len(self.actions)))

    def run(self, alpha=0.1, gamma=0.9, epsilon=0.6, epsilon_decay=0.2, episodes=1000):
        for episode in range(episodes):
            if episode%100 == 0:
                print(f"Episode {episode}")
            # generated initial state shouldn't take into account burned places
            state = random.randint(0,len(self.states)-1)
            while self.states[state] > 400:
                state -=1
            total_fire = 400
            while total_fire > 0:
                if np.random.rand() < epsilon:
                    action = np.random.randint(0,len(self.actions)-1)
                else:
                    action = np.argmax(self.q_table[state])
                    
                next_state, reward, total_fire = self.take_action(state, action)
                self.q_table[state, action] = (1 - alpha) * self.q_table[state, action] + \
                                alpha * (reward + gamma * np.max(self.q_table[next_state]))
                state = next_state
            epsilon *= epsilon_decay

    def take_action(self, state, action):
        if state == 0:
            fire_list = self.generate_fire_list(1, self.states[state])
        else:
            fire_list = self.generate_fire_list(self.states[state-1]+1, self.states[state])
        model = FireControllerAgentModel(width=self.width, height=self.height, 
                                         fire_fighters=self.fire_fighters, fire_agents_values=fire_list)
        next_state_min_range = self.states[state+1]+1 if state<len(self.states)-1 else self.states[state]
        while True:
            model.step()
            total_fire = model.get_total_fire_level()
            if total_fire == 0:
                next_state = 0
                break
            if total_fire >= next_state_min_range:
                next_state = state+1 if state<len(self.states)-1 else state
                break
        reward = self.fire_fighters-self.actions[action]-total_fire-model.get_total_burned_level()
        return next_state, reward, total_fire
                
    def generate_fire_list(self, range_start, range_end, max_length=100):
        digit_list = []
        
        for i in range(max_length):
            digit = random.randint(1, 4)
            digit_list.append(digit)
            if sum(digit_list) >= range_end:
                digit_list.pop()
                break
            if range_start < sum(digit_list) <= range_end:
                if random.random()<0.3:
                    break
        i = 0
        while sum(digit_list) < range_start+1:
            new_val = 4-digit_list[i]
            digit_list[i] += new_val
            i += 1
        return digit_list
        

In [7]:
q = QLearning()
q.run(episodes=1000, epsilon_decay=1, epsilon=0.65)

Episode 0
Episode 100
Episode 200
Episode 300
Episode 400
Episode 500
Episode 600
Episode 700
Episode 800
Episode 900


In [8]:
q.q_table

array([[ 2.82230235e+01,  4.19300214e+00,  1.60908633e+01,
         7.58670406e+00,  6.85374640e+00,  3.64444660e+00,
         3.69300214e+00,  2.20068945e+00,  7.36047930e+00,
         0.00000000e+00,  0.00000000e+00,  3.19300214e+00,
         0.00000000e+00,  0.00000000e+00,  1.97885507e+00,
         0.00000000e+00,  5.76000639e+00,  2.29439051e+00,
         4.82251408e+00,  0.00000000e+00],
       [ 5.20233328e+00,  6.56726527e+00,  3.64281869e+00,
         4.14007211e+00,  3.89300214e+00,  2.85438861e+01,
         3.39439051e+00,  2.03807015e+00,  1.93807015e+00,
         0.00000000e+00,  6.25670406e+00,  2.89439051e+00,
         7.70083636e+00,  3.14007211e+00,  5.00000000e-01,
         1.23807015e+00,  1.13807015e+00,  2.74007211e+00,
         2.73397898e+00,  0.00000000e+00],
       [ 4.44007211e+00,  2.47380823e+00,  4.24007211e+00,
         2.27380823e+00,  0.00000000e+00,  2.66965618e+01,
         3.39439051e+00,  3.59300214e+00,  3.49300214e+00,
         0.00000000e+00,  9.0

In [20]:
class TrainedModel(FireControllerAgentModel):
    def __init__(self, fire_agents_values: list, q_table):
        self.width = 10
        self.height = 10
        self.q_table = q_table
        self.grid = mesa.space.MultiGrid(self.width, self.height, False)
        self.schedule = mesa.time.RandomActivation(self)
        self.actions = [i+1 for i in range(20)]
        self.states = [20*(i+1) for i in range(25)]
        
        total_cells = self.width * self.height
        all_indices = [(i, j) for i in range(self.width) for j in range(self.height)]
        fire_cells = random.sample(all_indices, len(fire_agents_values))
        no_fire_cells = [index for index in all_indices if index not in fire_cells]
        for i, index in enumerate(fire_cells):
            a = FireAgent(new_uuid(), self, fire_agents_values[i])
            self.grid.place_agent(a, index)
            self.schedule.add(a)
        for i, index in enumerate(no_fire_cells):
            a = FireAgent(new_uuid(), self, 0)
            self.grid.place_agent(a, index)
            self.schedule.add(a)


        self.fire_fighters = self.calculate_next_firefighters()
        self.assign_firefighters_to_grid()

    def calculate_next_firefighters(self):
        fire_level = self.get_total_fire_level()
        state = None
        if fire_level == 0:
            fire_fighters = 0
        else:
            for i, value in enumerate(self.states):
                if i == 0:
                    if 0 < fire_level <= value:
                        state = i
                        break
                else:
                    if self.states[i-1] < fire_level <= value:
                        state = i
                        break
            fire_fighters = np.argmax(self.q_table[state])+1
        return fire_fighters
        
    def step(self):
        self.schedule.step()
        self.fire_fighters = self.calculate_next_firefighters()
        self.assign_firefighters_to_grid()
    

In [21]:
trained_model = TrainedModel([2,2,4,4,3,3,2,2,4,1,1,3,4,4,4], q.q_table)

while trained_model.get_total_fire_level() > 0:
    trained_model.step()
    print(f"total fire = {trained_model.get_total_fire_level()}, firefighters {trained_model.fire_fighters}")

print(f"Burned {starter_model.get_total_burned_level()}")

total fire = 38, firefighters 6
total fire = 41, firefighters 6
total fire = 40, firefighters 6
total fire = 43, firefighters 6
total fire = 46, firefighters 6
total fire = 45, firefighters 6
total fire = 48, firefighters 6
total fire = 49, firefighters 6
total fire = 45, firefighters 6
total fire = 45, firefighters 6
total fire = 45, firefighters 6
total fire = 45, firefighters 6
total fire = 45, firefighters 6
total fire = 45, firefighters 6
total fire = 41, firefighters 6
total fire = 41, firefighters 6
total fire = 41, firefighters 6
total fire = 38, firefighters 6
total fire = 38, firefighters 6
total fire = 40, firefighters 6
total fire = 38, firefighters 6
total fire = 36, firefighters 6
total fire = 36, firefighters 6
total fire = 33, firefighters 6
total fire = 33, firefighters 6
total fire = 33, firefighters 6
total fire = 33, firefighters 6
total fire = 33, firefighters 6
total fire = 33, firefighters 6
total fire = 30, firefighters 6
total fire = 28, firefighters 6
total fi